# Saving and Loading State

In [1]:
import asyncio
from autogen_core import CancellationToken
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [3]:
from typing import Text
from autogen_agentchat.conditions import MaxMessageTermination

agent_1 = AssistantAgent(
    name='Writer_1',
    model_client=model_client,
    system_message="You are a helpful assistant. Give the output in less than 30 words",
)

agent_2 = AssistantAgent(
    name='Writer_2',
    model_client=model_client,
    system_message="You are a helpful assistant.Give the output in less than 30 words",
)

terminationCondition = MaxMessageTermination(max_messages=3) 

agent_team = RoundRobinGroupChat(participants=[agent_1, agent_2],termination_condition=terminationCondition)

In [4]:
from autogen_agentchat.ui import Console
stream = agent_team.run_stream(task = 'Write a poem about sea in 3 lines')

await Console(stream)

---------- TextMessage (user) ----------
Write a poem about sea in 3 lines
---------- TextMessage (Writer_1) ----------
Vast and deep blue tide,  
Whispers secrets to the moon,  
Endless dreams abide.
---------- TextMessage (Writer_2) ----------
Endless blue and swell,  
Secrets dance beneath the waves,  
Quiet tales to tell.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a poem about sea in 3 lines', type='TextMessage'), TextMessage(source='Writer_1', models_usage=RequestUsage(prompt_tokens=36, completion_tokens=21), metadata={}, content='Vast and deep blue tide,  \nWhispers secrets to the moon,  \nEndless dreams abide.', type='TextMessage'), TextMessage(source='Writer_2', models_usage=RequestUsage(prompt_tokens=65, completion_tokens=19), metadata={}, content='Endless blue and swell,  \nSecrets dance beneath the waves,  \nQuiet tales to tell.', type='TextMessage')], stop_reason='Maximum number of messages 3 reached, current message count: 3')

In [5]:
team_state = await agent_team.save_state()

In [9]:
await agent_team.reset()

In [10]:
stream = agent_team.run_stream(task = 'What was the last line of the poem you wrote ')

await Console(stream)

---------- TextMessage (user) ----------
What was the last line of the poem you wrote 
---------- TextMessage (Writer_1) ----------
I'm sorry, I can't retrieve specific or past outputs like a poem I might have written. Can I help you with anything else?
---------- TextMessage (Writer_2) ----------
I don't have memory of specific poems I wrote. How can I assist you further?


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What was the last line of the poem you wrote ', type='TextMessage'), TextMessage(source='Writer_1', models_usage=RequestUsage(prompt_tokens=38, completion_tokens=26), metadata={}, content="I'm sorry, I can't retrieve specific or past outputs like a poem I might have written. Can I help you with anything else?", type='TextMessage'), TextMessage(source='Writer_2', models_usage=RequestUsage(prompt_tokens=72, completion_tokens=17), metadata={}, content="I don't have memory of specific poems I wrote. How can I assist you further?", type='TextMessage')], stop_reason='Maximum number of messages 3 reached, current message count: 3')

In [11]:
await agent_team.load_state(team_state)
stream = agent_team.run_stream(task = 'What was the last line of the poem you wrote ')

await Console(stream)

---------- TextMessage (user) ----------
What was the last line of the poem you wrote 
---------- TextMessage (Writer_1) ----------
Endless dreams abide.
---------- TextMessage (Writer_2) ----------
Endless dreams abide.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What was the last line of the poem you wrote ', type='TextMessage'), TextMessage(source='Writer_1', models_usage=RequestUsage(prompt_tokens=104, completion_tokens=5), metadata={}, content='Endless dreams abide.', type='TextMessage'), TextMessage(source='Writer_2', models_usage=RequestUsage(prompt_tokens=117, completion_tokens=5), metadata={}, content='Endless dreams abide.', type='TextMessage')], stop_reason='Maximum number of messages 3 reached, current message count: 3')

In [ ]:
# TODO : write and load from a json file

import json